In [ ]:
# This reloads python code imported from *.py files automatically
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tqdm 
import tqdm.notebook

from optimization import *

import time
# optimal solution
from optimal_solution import *
from clustering import *

In [ ]:
from scipy.cluster.vq import kmeans,vq,whiten
import itertools

### Clustering tests

In [ ]:
n_iter = 5000

# parameters
l = 0.5 # lambda in [0,1], or in [0,2] for G2
beta = 50

# initialization
N = 10
n_selected = 0
g = G1(N)
g = G2(N)
citiesX = g.x
citiesV = g.v

#### Simple clusters

* Automatic selection of centroids

In [ ]:
start_time = time.time()
data = whiten(g.x)
centroids,_ = kmeans(data,10)
clx,_ = vq(data,centroids)
centroids_V=[np.sum(g.v, where=(clx == k)) for k in range (centroids.shape[0])]
d1=(time.time() - start_time)

fig,axes=plt.subplots(1,2,figsize=(12,4))
axes[0].scatter(data[:,0],data[:,1], s=g.v,c=clx ,label='cities')
sequence = np.arange(centroids.shape[0])
axes[1].scatter(centroids[:, 0], centroids[:, 1],s=centroids_V, c=sequence,label='centroids')
print(np.sum(g.v))
print(np.sum(centroids_V))
print(d1)

* with defined centroids: population is involved

In [ ]:
start_time = time.time()
ind = np.argpartition(g.v, -10)[-10:]
data = whiten(g.x)
centroids=(data[ind])
clx,_ = vq(data,centroids)
centroids_V=[np.sum(g.v, where=(clx == k)) for k in range (centroids.shape[0])]
d1=(time.time() - start_time)

fig,axes=plt.subplots(1,2,figsize=(12,4))
axes[0].scatter(data[:,0],data[:,1], s=g.v,c=clx ,label='cities')
sequence = np.arange(centroids.shape[0])
axes[1].scatter(centroids[:, 0], centroids[:, 1], s=centroids_V, c=sequence,label='centroids')
print(np.sum(g.v))
print(np.sum(centroids_V))

print(d1)

### Study case 10 cities

In [ ]:
n_iter = 5000
# parameters
l = 0.5 # lambda in [0,1], or in [0,2] for G2
beta = 20
# initialization
N = 10
n_selected = 0
g = G1(N)
# g = G2(N)

result=bruteforce_sol(N, l, g)
print(result)
best_selected_cities=np.array(result[0])
brute_force_f, max_dist, max_idx, convex_hull = objective_function_(N, l, g, None, best_selected_cities, None, None)
print(brute_force_f)
print(result[1])
brute_force_f=result[1]
plotResult(g,0,best_selected_cities,None,np.zeros(n_iter) +brute_force_f,None,None)

In [ ]:
best_sol, best_f = opt_solution_circle(N, l, g, verbose=True)
print(f"Best solution: {best_f}")    
plotResult(g,0, best_sol, None, np.zeros(n_iter) + best_f, None, None)  

In [ ]:
initial_selection_probability=0.5

In [ ]:
np.random.seed()
selected_cities=(np.random.rand(N) <= initial_selection_probability).astype(np.int32)
# selected_cities=np.zeros(N)
print(selected_cities)

f_st=np.zeros((5,n_iter))
selected_st=np.zeros((5,N))
fig=plt.plot
for i in range(4):
    selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize_with_initialize_betas(g, l, selected_cities, betas=[5,10,20,50,100], 
                                              n_iter=n_iter,mutation_strategy=i, initial_selection_probability=initial_selection_probability,
                                              precompute_pairwise_dist=False, verbose=False)
    
    f = objective_function_simple(N, l, g, selected_cities_n_convex,
                          pairwise_distances=None)
    loss_value_convex = f
    if type(selected_cities_n) == list:
        selected_cities_n = selected_cities_n[-1]
    loss_values[-1]=(loss_value_convex) 
#     print(loss_value_convex)
    f_st[i]=loss_values
    plt.plot(f_st[i],label = "mutation_strategy="+str(i))
    selected_st[i]=selected_cities_n_convex
    
#combining 3 and 2    
selected_cities_n, selected_cities_n_convex, loss_values1,loss_value_convex = optimize_with_initialize_betas(g, l,selected_cities, betas=[5,10,20,50,100],  n_iter=n_iter,mutation_strategy=3, precompute_pairwise_dist=False, verbose=False)
loss_values1[-1]=(loss_value_convex)  
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize_with_initialize_betas(g, l, selected_cities_n_convex, betas=[50],n_iter=n_iter,mutation_strategy=2,precompute_pairwise_dist=False, verbose=False)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
loss_values[-1]=(loss_value_convex) 
total_loss=np.concatenate((loss_values1,loss_values))
total_loss=total_loss[::2]
f_st[4]=total_loss
selected_st[4]=selected_cities_n_convex
plt.plot(f_st[4],label = "Combining")

# plt.hlines(best_f, 0, n_iter, linestyles="-.", label="optimal solution")
plt.hlines(result[1], 0, n_iter, linestyles="--", label="brut-force solution")

plt.legend()
print(np.amin(f_st,1))
print(np.amin(f_st))
best_min=np.amin(f_st)
best=np.argmin(np.amin(f_st,1))
print("The best cost is=%f, found for strategy %i"%(best_min,best))
# print(f'Approximated Bruteforce best solution {best_f}')
print(f'Real Bruteforce best solution {brute_force_f}')

plotResult(g,0,selected_st[best],None,f_st[best],None,None)

if f_st[best][-1]==brute_force_f:
    print("Best solution found!")



In [ ]:
print(selected_st[best])
f, max_dist, max_idx, convex_hull = objective_function_(N, l, g, None, selected_st[best], None, None)
print(f)
f, max_dist, max_idx, convex_hull = objective_function_(N, l, g, None, best_selected_cities, None, None)
print(f)
